In [ ]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
#import pyarrow as pa
#import pyarrow.parquet as pq
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import StratifiedKFold
#import lightgbm as lgb
#from lightgbm import LGBMClassifier
from tqdm import tqdm

In [ ]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [ ]:
to_train = pd.read_pickle('phase_1.pkl')

In [ ]:
for col in df.columns:
    if str(type(df[col][0])) == "<class 'numpy.float64'>":
        df[col] = df[col].astype(np.float16)

In [ ]:
gc.collect()

In [ ]:
to_train = df.dropna()
to_train = to_train[[x for x in to_train.columns if (x != 'customer_ID' and x != 'S_2')]]
print(to_train.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(to_train[to_train.columns[:-1]].values, to_train['target'].values, test_size=0.33, random_state=42)


In [ ]:
print('Train data shape: ' + str(X_train.shape)) 
print('Test data shape: ' + str(X_test.shape)) 

In [ ]:
def feature_importance(model):
    n_features = 15 #X_train.shape[1]
    plt.figure(figsize = (25,25))
    plt.barh(range(n_features), model.feature_importances_.sort()[:n_features], align = 'center')
    plt.yticks(np.arange(n_features), to_train.columns[:n_features])
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

In [ ]:
amex_metric(pd.DataFrame({'target' : y_test}), pd.DataFrame({'prediction' : tree.predict(X_test)}))